### Import the library

In [6]:
import torch
import pickle
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch_geometric.nn as gnn

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

In [7]:
from net import PSOGNN
from pso import PSO 
from base_function import *

### Data Preparation

In [8]:
pickle_path = r'/Users/admin/Code/deepso/parameter_dataset.pkl'
with open(pickle_path, 'rb') as f:
    all_functions_by_dim = pickle.load(f)

In [9]:
print(all_functions_by_dim[1]['ackley'][0])
print(all_functions_by_dim[1]['ackley'])

[20.96273764529804, 0.11035671419673816, 4.918649250038615]
[[20.96273764529804, 0.11035671419673816, 4.918649250038615], [23.74530149501887, 0.16871914789893266, 6.524365116229994], [16.771837051955302, 0.26254914745365177, 7.501011305001985], [23.76139950018486, 0.12267042879300734, 7.764721074982338], [23.149094492090082, 0.2806583324321734, 7.020621596066444], [19.946842976168938, 0.2373275682401191, 5.3354023818848795], [20.138485163521086, 0.1595125285229236, 5.846546005015856], [17.153966051085295, 0.18172931011202878, 5.326939603577901], [21.185834221271215, 0.136743206287994, 6.429986859348397], [18.276672079344234, 0.16605048078400958, 7.217652999240284], [19.095220969158703, 0.28486193801473003, 5.157505115471498], [20.38939602338722, 0.2817044427509823, 4.835163085284753], [17.855669479733944, 0.2795813724581502, 7.001885012077157], [15.736380816196823, 0.1324005557979872, 6.206276583880579], [22.912606655554015, 0.2602500232413104, 4.795516524601839], [18.50005620705775, 0

In [10]:
train_data_combined = []
test_data_combined = []

for d, func_dict in all_functions_by_dim.items():
    for func_type, func_list in func_dict.items():
        train_funcs, test_funcs = train_test_split(func_list, test_size=0.3, random_state=42)

        train_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in train_funcs])
        test_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in test_funcs])

print('split done')


split done


In [11]:
print(train_data_combined[1])

{'dim': 1, 'func_type': 'ackley', 'params': [20.081788668511855, 0.24612029062900076, 6.997327583270227]}


In [12]:
splited_data = {'train': train_data_combined, 'test': test_data_combined}
splited_path = r'/Users/admin/Code/deepso/splited_data.pkl'
with open(pickle_path, 'wb') as f:
    pass
with open(splited_path, 'wb') as f:
    pickle.dump(splited_data, f)

print(f"All functions split into combined train and test sets and saved to '{splited_path}'")

In [13]:
def create_batches(dataset, batch_size): 
    random.shuffle(dataset)
    for i in range(0, len(dataset), batch_size): 
        batch = dataset[i:i + batch_size]
        yield batch

### Train

In [25]:
batch_size = 16
mini_batch_size = 4
num_epochs = 10

UB = 1
LB = 0
num_particles = 100

In [23]:

def train(dataloader, accumulate_step):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        function_loader = create_batches(dataloader, batch_size)

        # optimizer.zero_grad()

        for batch in function_loader:
            # print(batch)
            for i in range(0, len(batch), mini_batch_size):
                mini_batch = batch[i : i + mini_batch_size]
                # print('mini_batch', mini_batch)
                for func in mini_batch: 
                    print('func',func)

                    # break
                    func_type = next(iter(func.keys()))
                    params = func[func_type]
                    def function(x):
                        function_instance = Function.get_function(func_type, x, params)
                        return function_instance.evaluate_function()
                    
                    X = torch.rand(num_particles, dim)

                    Pso = PSO(X, function, LB, UB)
                    position_best, best, mean_fitness = Pso.run()

                    loss = mean_fitness
                    loss = loss/accumulate_step
                    loss.backward()
                    if (i+1)%accumulate_step == 0:
                        optimizer.step()
                        optimizer.zero_grad()




In [ ]:
# model = PSOGNN(node_input_dim=dim)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
accumulate_step = 4
train(train_data_combined, accumulate_step=accumulate_step)

In [31]:
def evaluate(model, dataloader, num_particles, dim, LB, UB):
    model.eval()  
    total_loss = 0.0  
    count = 0  

    with torch.no_grad():  
        for batch in dataloader:
            for i in range(0, len(batch), mini_batch_size):
                mini_batch = batch[i : i + mini_batch_size]

                for func_type, params in mini_batch:
                    def evaluate_function(x):
                        function_instance = Function.get_function(func_type)
                        return function_instance.evaluate_function()

                    X = torch.rand(num_particles, dim)
                    edge_index = torch.combinations(torch.arange(num_particles), r=2).t().contiguous()
                    data = Data(x=X, edge_index=edge_index)

                    param = model(data)
                    W, C1, C2 = param[:, 0], param[:, 1], param[:, 2]

                    Pso = PSO(num_particles, evaluate_function, dim, LB, UB, W, C1, C2, X)
                    position_best, best, mean_fitness = Pso.run()

                    total_loss += mean_fitness
                    count += 1

    average_loss = total_loss / count if count > 0 else float('inf')
    print(f"Average loss (mean fitness) on evaluation: {average_loss:.4f}")

    return average_loss